이 장에서는 스타벅스의 입지 전략에 대한 가서을 세우고 증명해보는 과정을 갖겠습니다.
1. 거주 인구가 많은 지역에 스타벅스 매자이 많이 입지해 있을 것이다.
2. 직장인이 많은 지역에 스타벅스 매장이 많이 입지해 있을 것이다.

실제로 입지 분석에는 매우 어렵고 복잡한 데이터 분석 기법들이 필요하지만,  
이 책에서는 쉽게 수집할 수 있는 데이터들을 이용해 시각화하는 것만으로도 주요한 인사이트를 얻을 수 있다는 것을 보여드리고자 합니다.

### 5.1 데이터 수집
1. 서울시 내에 출점한 스타벅스들의 위치를 파악해야 합니다.
2. 두 가지 가설을 검증하기 위해서 인구통계 데이터 수집도 필요

In [1]:
#서울시 내에 스타벅스들의 위치 파악을 위한 필요한 모듈
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
#크롬 드라이버를 불러오고 스타벅스 매장 url을 엽니다.
driver = webdriver.Chrome('../chromedriver.exe')
url = 'https://www.starbucks.co.kr/store/store_map.do?disp=locale'
driver.get(url)

SessionNotCreatedException: Message: session not created: This version of ChromeDriver only supports Chrome version 80


In [5]:
#서울 버튼을 클릭합니다.
seoul_btn = 'a.set_sido_cd_btn'
driver.find_element_by_css_selector(seoul_btn).click()

In [6]:
#서울 아래 전체를 클릭합니다.
all_btn = 'a.set_gugun_cd_btn'
driver.find_element_by_css_selector(all_btn).click()

In [7]:
#BeautifulSOup 으로 HTML 파서 만들기
html = driver.page_source
soup = BeautifulSoup(html,'html.parser')

In [8]:
#위치 정보 태그를 파악한 후 모두 찾아오기
starbucks_soup_list = soup.select('li.quickResultLstCon')
print(len(starbucks_soup_list))

513


In [9]:
#태그 문자열 살펴보기
starbucks_soup_list[0]

<li class="quickResultLstCon" data-code="3762" data-hlytag="null" data-index="0" data-lat="37.501087" data-long="127.043069" data-name="역삼아레나빌딩" data-storecd="1509" style="background:#fff"> <strong>역삼아레나빌딩  <img alt="" class="setStoreFavBtn mCS_img_loaded" data-my_siren_order_store_yn="N" data-name="역삼아레나빌딩" data-store="1509" data-yn="N" src="//image.istarbucks.co.kr/common/img/store/icon_fav_off.png"/></strong> <p class="result_details">서울특별시 강남구 언주로 425 (역삼동)<br/>1522-3232</p> <i class="pin_general">리저브 매장 2번</i></li>

태그 문자열을 통해 매장 이름(data-name), 주소(p.result_detailes), 위도(data-lat), 경도(dat-long) 등의 정보를 알 수있ㅅ브니다.

In [22]:
# 스타벅스 매장 정보 샘플 확인
starbucks_store = starbucks_soup_list[0]
name = starbucks_store.select('strong')[0].text.strip()
lat = starbucks_store['data-lat'].strip()
lng = starbucks_store['data-long'].strip()
store_type = starbucks_store.select('i')[0]['class'][0][4:]
address = str(starbucks_store.select('p.result_details')[0]).split('<br/>')[0].split('>')[1]
tel = str(starbucks_store.select('p.result_details')[0]).split('<br/>')[1].split('<')[0]

print(name)
print(lat)
print(lng)
print(store_type)
print(address)
print(tel)


역삼아레나빌딩
37.501087
127.043069
general
서울특별시 강남구 언주로 425 (역삼동)
1522-3232


In [29]:
#서울시 스타벅스 매장 목록 데이터 만들기
starbucks_list = []
for starbucks_store in starbucks_soup_list:
    name = starbucks_store.select('strong')[0].text.strip()
    lat = starbucks_store['data-lat'].strip()
    lng = starbucks_store['data-long'].strip()
    store_type = starbucks_store.select('i')[0]['class'][0][4:]
    address = str(starbucks_store.select('p.result_details')[0]).split('<br/>')[0].split('>')[1]
    tel = str(starbucks_store.select('p.result_details')[0]).split('<br/>')[1].split('<')[0]

    starbucks_list.append([name,lat,lng,store_type,address,tel])

starbucks_list[:3]

[['역삼아레나빌딩',
  '37.501087',
  '127.043069',
  'general',
  '서울특별시 강남구 언주로 425 (역삼동)',
  '1522-3232'],
 ['논현역사거리',
  '37.510178',
  '127.022223',
  'general',
  '서울특별시 강남구 강남대로 538 (논현동)',
  '1522-3232'],
 ['신사역성일빌딩',
  '37.514132',
  '127.020563',
  'general',
  '서울특별시 강남구 강남대로 584 (논현동)',
  '1522-3232']]

In [31]:
# pandas 데이터프레임 생성
columns = ['매장명','위도','경도','매장타입','주소','전화번호']
seoul_starbucks_df = pd.DataFrame(starbucks_list,columns=columns)
seoul_starbucks_df.head()

,매장명,위도,경도,매장타입,주소,전화번호
0,역삼아레나빌딩,37.501087,127.043069,general,서울특별시 강남구 언주로 425 (역삼동),1522-3232
1,논현역사거리,37.510178,127.022223,general,서울특별시 강남구 강남대로 538 (논현동),1522-3232
2,신사역성일빌딩,37.514132,127.020563,general,서울특별시 강남구 강남대로 584 (논현동),1522-3232
3,국기원사거리,37.499517,127.031495,general,서울특별시 강남구 테헤란로 125 (역삼동),1522-3232
4,스탈릿대치R,37.494668,127.062583,reserve,서울특별시 강남구 남부순환로 2947 (대치동),1522-3232


In [32]:
#데이터프레임 요약 정보 확인
seoul_starbucks_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 513 entries, 0 to 512
Data columns (total 6 columns):
매장명     513 non-null object
위도      513 non-null object
경도      513 non-null object
매장타입    513 non-null object
주소      513 non-null object
전화번호    513 non-null object
dtypes: object(6)
memory usage: 24.1+ KB


In [33]:
seoul_starbucks_df.to_excel('./files/0416seoul_starbucks_list.xlsx')